In [111]:
#Import Dependencies
import pandas as pd

In [112]:
# The path to our CSV file
school_file = "Resources/schools_complete.csv"

# Read our Students data into pandas
df_school = pd.read_csv(school_file)
df_school.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [113]:
# The path to our CSV file
student_file = "Resources/students_complete.csv"

# Read our Students data into pandas
df_student = pd.read_csv(student_file)
df_student.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [114]:
# Merging  student with school file
all_df = pd.merge(df_school,df_student, on=('school_name'))
all_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


### District Summary

* Create a high level snapshot (in table form) of the district's key metrics, including:
  * Total Schools
  * Total Students
  * Total Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)


In [115]:
#Total Schools
total_schools = all_df["school_name"].nunique()
#Total Students
total_students = all_df["Student ID"].count()
#Total Budget
total_budget = df_school["budget"].sum()
#Average Math Score
avg_math = all_df["math_score"].mean()
#Average Reading Score
avg_read = all_df["reading_score"].mean()
#% Passing Math
math_pass = df_student.loc[df_student["math_score"] >= 70]
count_pass_math = math_pass["math_score"].count()
per_math_pass = (count_pass_math/total_students)*100
#% Passing Reading
read_pass = df_student.loc[df_student["reading_score"] >= 70]
count_pass_read = read_pass["reading_score"].count()
per_read_pass = (count_pass_read/total_students)*100
#Overall Passing Rate (Average of the above two)
overall=(per_math_pass + per_read_pass )/2
# Make dataframe with the results
district_summary = pd.DataFrame({"Total Schools" : [total_schools],
                   "Total Students" : [total_students],
                   "Total Budget" : [total_budget],
                   "Average Math Score" :[avg_math],
                   "Average Reading Score" : [avg_read],
                    "% Passing Math" : [per_math_pass],
                   "% Passing Reading" : [per_read_pass],
                   "% Overall Passing" : [overall]
                  })



district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


### School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)


In [126]:

# Set new index to school_name
#all_df = all_df.set_index("school_name")
# Determine the school type
school_types = all_df.set_index(["school_name"])["type"]

# Calculate total student count per school
school_students = all_df["school_name"].value_counts()

# Calculate per school and per student budget
school_budget = all_df.groupby(["school_name"]).mean()["budget"]
perstudent_budget = school_budget/ school_students

# calculate average math and reading scores
Avg_math = all_df.groupby(["school_name"]).mean()["math_score"]
Avg_read = all_df.groupby(["school_name"]).mean()["reading_score"]

# Count of schools passing Math and Reading
math_pass_count =  all_df[all_df["math_score"] > 70].groupby("school_name").count()["student_name"]
reading_pass_count =  all_df[all_df["reading_score"] > 70].groupby("school_name").count()["student_name"]
# calculate percentages:
percent_passing_math = math_pass_count / school_students * 100
percent_passing_reading = reading_pass_count / school_students * 100
overall_passing = (percent_passing_math + percent_passing_reading) / 2
# Convert to data frame
School_Summary = pd.DataFrame({"Type":school_types, "Total Students":school_students,
                               "Total School Budget":school_budget, "Per Student Budget":perstudent_budget,
                                "Average Math Score":Avg_math, "Average Reading Score":Avg_read,
                                 "% Passing Math":math_pass_count, "% Passing Reading":reading_pass_count,
                                 "Overall Passing Rate":overall_passing})
School_Summary = School_Summary[["Type","Total Students","Total School Budget","Per Student Budget",
                                "Average Math Score","Average Reading Score","% Passing Math","% Passing Reading",
                                 "Overall Passing Rate"]]

School_Summary["Total School Budget"] = School_Summary["Total School Budget"].map("${:,.2f}".format)
School_Summary["Per Student Budget"] = School_Summary["Per Student Budget"].map("${:,.2f}".format)

School_Summary


ValueError: cannot reindex from a duplicate axis